In [ ]:
import pandas as pd
import os

In [15]:
#Buscando a base dados estatísticos no sita da ANAC
#Essa base contem informações diversas do setor: n° de pax, carga, distancia, decolagens, etc.
url = "https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Dados%20Estat%C3%ADsticos%20do%20Transporte%20A%C3%A9reo/Dados_Estatisticos.csv"
dados_estatisticos = pd.read_csv(url, encoding='utf-8', delimiter=';', skiprows=1) 

In [ ]:
#Filtrando somente os dados de 2024
base_aerea = dados_estatisticos[dados_estatisticos["ANO"] == 2024]
base_aerea.columns

Index(['EMPRESA_SIGLA', 'EMPRESA_NOME', 'EMPRESA_NACIONALIDADE', 'ANO', 'MES',
       'AEROPORTO_DE_ORIGEM_SIGLA', 'AEROPORTO_DE_ORIGEM_NOME',
       'AEROPORTO_DE_ORIGEM_UF', 'AEROPORTO_DE_ORIGEM_REGIAO',
       'AEROPORTO_DE_ORIGEM_PAIS', 'AEROPORTO_DE_ORIGEM_CONTINENTE',
       'AEROPORTO_DE_DESTINO_SIGLA', 'AEROPORTO_DE_DESTINO_NOME',
       'AEROPORTO_DE_DESTINO_UF', 'AEROPORTO_DE_DESTINO_REGIAO',
       'AEROPORTO_DE_DESTINO_PAIS', 'AEROPORTO_DE_DESTINO_CONTINENTE',
       'NATUREZA', 'GRUPO_DE_VOO', 'PASSAGEIROS_PAGOS', 'PASSAGEIROS_GRATIS',
       'CARGA_PAGA_KG', 'CARGA_GRATIS_KG', 'CORREIO_KG', 'ASK', 'RPK', 'ATK',
       'RTK', 'COMBUSTIVEL_LITROS', 'DISTANCIA_VOADA_KM', 'DECOLAGENS',
       'CARGA_PAGA_KM', 'CARGA_GRATIS_KM', 'CORREIO_KM', 'ASSENTOS', 'PAYLOAD',
       'HORAS_VOADAS', 'BAGAGEM_KG'],
      dtype='object')

In [65]:
base_aerea_filtrada = base_aerea[base_aerea['AEROPORTO_DE_ORIGEM_PAIS'] == 'BRASIL']

#Criação da variável de quantidade de decolagens por aeroporto
decolagens = pd.DataFrame(base_aerea_filtrada.groupby('AEROPORTO_DE_ORIGEM_SIGLA')['DECOLAGENS'].sum())

#Variável de qunatidade de voos diretos
mercados_diretos = pd.DataFrame(base_aerea_filtrada.groupby('AEROPORTO_DE_ORIGEM_SIGLA')['AEROPORTO_DE_DESTINO_SIGLA'].count())


Tentativa fazer o download das planilhas de tarifa diretamente do site da ANAC

In [ ]:
pip install selenium beautifulsoup4


In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  # Importar a classe Options corretamente
import time

In [ ]:
chrome_options = Options()  
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

chrome_driver_path = "C:/Users/GUilherme.costa/Downloads/chromedriver-win32/chromedriver-win32/chromedriver.exe"
               
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)


try:
    driver.get("https://sas.anac.gov.br/sas/downloads/view/frmDownload.aspx?tema=14")
    
    time.sleep(5)
    
    select_ano = Select(driver.find_element(By.ID, "MainContet_listAno"))
    select_ano.select_by_visible_text("2024")
    
    select_tema = Select(driver.find_element(By.ID, "MainContent_listTema"))  
    select_tema.select_by_visible_text("Tarifas Transporte Aéreo Passagerios Domésticos")
    
    botao_buscar = driver.find_element(By.ID, "MainContent_btnListaArquivos")  
    botao_buscar.click()
    
    time.sleep(5)
    
    botao_selecionartodos = driver.find_element(By.ID, "MainContent_btnMarcar")
    botao_selecionartodos.click()

    botao_download = driver.find_element(By.ID, "MainContent_btnBaixar")
    botao_download.click()
    
    time.sleep(10)  

finally:
    driver.quit()


In [139]:
#O código acima ainda está dando erro. Acredito que estou com problema para acessar no site da Anac. Ainda pretendo voltar aqui para automatizar
#Enquanto isso, vou fazer download dos arquivos na mão, como um boomer

caminho_pasta = "C:/Users/GUilherme.costa/Downloads/anac_14925995"
arquivos_csv = [arquivo for arquivo in os.listdir(caminho_pasta) if arquivo.lower().endswith('.csv')]

dataframes = []

for arquivo in arquivos_csv:
    caminho_arquivo = os.path.join(caminho_pasta, arquivo)
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', delimiter=';') 
    dataframes.append(df)

base_tarifa = pd.concat(dataframes, ignore_index=True)

base_tarifa['TARIFA'] = (
    df['TARIFA']
    .str.replace('.', '', regex=False)  
    .str.replace(',', '.')              
    .apply(pd.to_numeric, errors='coerce') 
)


In [142]:
#Para o cálculo da tarifa não da pra fazer apenas uma média, é preciso ponderar por  assento por rota. Vou fazer isso para o ano
base_tarifa['OD'] = base_tarifa['ORIGEM'] +'-' + base_tarifa['DESTINO']
base_tarifa['TARIFA_ASSENTO'] = base_tarifa['TARIFA']*base_tarifa['ASSENTOS']  

#Cálculo do total de assentos por roda
soma_assentos_por_rota = base_tarifa.groupby('OD')['ASSENTOS'].sum().reset_index()
soma_assentos_por_rota.columns = ['OD', 'SOMA_ASSENTOS']

#Cálculo da soma das tarifas por rota
soma_tarifa_assentos = base_tarifa.groupby('OD')['TARIFA_ASSENTO'].sum().reset_index()
soma_tarifa_assentos.columns = ['OD', 'SOMA_TARIFA*ASSENTOS']

tarifa_ponderada = soma_assentos_por_rota.merge(soma_tarifa_assentos, on = 'OD' )
tarifa_ponderada['TARIFA_P'] = tarifa_ponderada['SOMA_TARIFA*ASSENTOS'] / tarifa_ponderada['SOMA_ASSENTOS']
tarifa_ponderada = tarifa_ponderada.drop(['SOMA_ASSENTOS',	'SOMA_TARIFA*ASSENTOS'], axis =1)

In [ ]:
#Juntando as duas bases. Reparar que a tarifa ponderada é a mesma para o mesmo OD naquele ano
base_tarifa = base_tarifa.merge(tarifa_ponderada, on='OD')
base_tarifa.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,OD,TARIFA_ASSENTO,TARIFA_P
0,2024,1,ABJ,SBSV,SDLO,750.0,8,SBSV-SDLO,6000.0,181.224839
1,2024,1,ABJ,SBSV,SDLO,950.0,8,SBSV-SDLO,7600.0,181.224839
2,2024,1,ABJ,SBSV,SDLO,1150.0,14,SBSV-SDLO,16100.0,181.224839
3,2024,1,ABJ,SBSV,SDLO,600.0,1,SBSV-SDLO,600.0,181.224839
4,2024,1,ABJ,SBSV,SIRI,700.0,11,SBSV-SIRI,7700.0,102.516109
